* 日收益率和对数收益率：用于衡量资产的短期收益和波动性。
* 价格范围百分比：用于评估资产在一天内的波动程度。
* 移动平均线：用于识别资产的趋势。
* 波动率：用于评估资产的风险水平。

### 1. Daily Return（日收益率）
业务含义：
日收益率表示资产在一天内收盘价的变化率，是衡量资产短期收益的指标。

- 计算方法：pct_change() 是 Pandas 中计算百分比变化的函数，它会计算当前值与前一个值的差值，然后除以前一个值，结果乘以 100 转换为百分比形式。

用途：
- 用于评估资产的短期表现，帮助投资者快速了解资产在一天内的增值或贬值情况。
- 可以用于计算资产的平均日收益率，进而估算其年化收益率。

### 2. Log Return（对数收益率）
业务含义：
对数收益率是使用对数差分计算的收益率，它与普通收益率相比，具有更好的数学性质，例如对称性和可加性。
- 计算方法：
df1['Close'] / df1['Close'].shift(1) 计算的是连续两天收盘价的比值。
np.lo() 对比值取自然对数，得到对数收益率。

用途：
- 对数收益率在数学上更稳定，适合用于时间序列分析和统计建模。
- 便于计算长期收益率的累积效果，因为对数收益率可以直接相加。
- 在金融模型中，对数收益率常用于计算资产的波动率和风险。

### 3. Range Percentage（价格范围百分比）
业务含义：
价格范围百分比表示当天最高价与最低价之间的波动幅度，反映了资产在一天内的价格波动范围。
- 计算方法：
df1['High'] - df1['Low'] 计算当天的价差。
将价差除以当天的最低价，再乘以 100，得到价格波动的百分比。

用途：
- 用于衡量资产在一天内的价格波动程度。
- 较高的 range_pct 表示资产价格在当天内波动较大，可能暗示市场活跃或存在不确定性。
- 交易者可以用这个指标来评估市场波动性，制定交易策略。

### 4. Moving Average（移动平均线）
业务含义：
移动平均线是技术分析中常用的指标，用于平滑价格数据，帮助识别趋势。
- 计算方法：
rolling(20) 表示以 20 天为窗口进行滚动计算。
.mean() 计算窗口内的平均值，得到 20 日移动平均线。

用途：
- 移动平均线可以帮助投资者判断资产的短期趋势。
- 当资产价格高于移动平均线时，可能表示资产处于上升趋势；反之则可能处于下降趋势。
- 交易者常使用移动平均线交叉策略来判断买卖时机。

### 5. Volatility（波动率）
业务含义：
波动率是衡量资产价格波动程度的指标，通常用于评估资产的风险。
- 计算方法：
rolling(20) 表示以 20 天为窗口进行滚动计算。
.std() 计算窗口内对数收益率的标准差，表示短期波动性。
np.sqrt(252) 将日波动率转换为年化波动率（假设一年有 252 个交易日）。

用途：
- 波动率越高，资产价格的不确定性越大，风险也越高。
- 投资者可以用波动率来评估资产的风险水平，或者将其与其他资产的波动率进行比较。
- 在投资组合管理中，波动率是衡量风险的重要指标，常用于优化投资组合的风险与收益平衡。


In [1]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['PingFang SC']  # 确保设置正确
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
from pandas_datareader import data, wb
import yfinance as yf
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import ta
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
%matplotlib inline

In [2]:
def plot_technical(df, ticker):
    #导入
    import matplotlib.pyplot as plt
    plt.rcParams['font.sans-serif'] = ['PingFang SC']  # 确保设置正确
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
    from pandas_datareader import data, wb
    import yfinance as yf
    import pandas_datareader as pdr
    import pandas as pd
    import numpy as np
    import os
    import datetime
    import matplotlib.pyplot as plt
    import seaborn as sns
    import ta
    import matplotlib.dates as mdates
    import matplotlib.gridspec as gridspec
    %matplotlib inline

    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # 确保设置正确
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

    plt.style.use('dark_background') #可视化风格
    fig = plt.figure(figsize=(18, 24))
    gs = gridspec.GridSpec(6, 1, height_ratios=[3, 1, 1, 1, 1, 1], hspace=0.8)

    # ------------------------------
    # 1. 价格趋势与均线（核心趋势）
    # ------------------------------
    ax1 = plt.subplot(gs[0])
    ax1.plot(df.index, df['Close'], label='Closing Price', color='#00FF7F', linewidth=2)
    ax1.plot(df.index, df['MA10'], label='10D MA (中期趋势线)', color='#FFD700', linestyle='--', linewidth=1.5)
    ax1.plot(df.index, df['MA50'], label='50D MA (长期趋势线)', color='#FF6347', linestyle='-.', linewidth=1.5)
    # 检测黄金/死亡交叉
    df['MA10_above_MA50'] = df['MA10'] > df['MA50']
    cross_points = df[df['MA10_above_MA50'].ne(df['MA10_above_MA50'].shift())].index
       # 在交叉点添加符号标注
    for date in cross_points:
       y_val = df.loc[date, 'MA10']
       color = 'gold' if df.loc[date, 'MA10'] > df.loc[date, 'MA50'] else 'red'
       ax1.scatter(date, y_val, s=80, color=color, edgecolors='white', zorder=5, 
                     marker='*' if color=='gold' else 'X')
    
    ax1.set_title(f'{ticker} - 价格趋势与均线分析', 
                 color='white', fontsize=12, pad=12)
    ax1.legend(loc='upper left')

    # ------------------------------
    # 2. 成交量与价格动能（量价分析）
    # ------------------------------
    ax2 = plt.subplot(gs[1], sharex=ax1)
    ax2.bar(df.index, df['Volume'], 
           color=np.where(df['Close'] > df['Open'], '#00FF9D', '#FF006E'), 
           alpha=0.8, width=0.8 )
    ax2.set_title('成交量分析\n[绿色量柱配合价格上涨为健康上涨，红色量柱伴随下跌需警惕]', 
                color='white', fontsize=12, pad=12)

    # ------------------------------
    # 3. RSI超买超卖指标（动量震荡）
    # ------------------------------
    ax3 = plt.subplot(gs[2], sharex=ax1)
    ax3.plot(df.index, df['RSI'], color='#00C8FF', linewidth=1.5)
    ax3.fill_between(df.index, 70, 90, color='#FF4D4D', alpha=0.15)
    ax3.fill_between(df.index, 30, 10, color='#4DFF4D', alpha=0.15)
    ax3.text(0.01, 0.95, '超买区域', transform=ax3.transAxes, color='#FF4D4D', fontsize=10)
    ax3.text(0.01, 0.05, '超卖区域', transform=ax3.transAxes, color='#4DFF4D', fontsize=10)
    ax3.set_title('RSI相对强弱指标 (14日周期)\n[70以上预警超买，30以下预示超卖，背离信号有效性更强]', 
                color='white', fontsize=12, pad=12)

    # ------------------------------
    # 4. 波动率指数（市场情绪）
    # ------------------------------
    ax4 = plt.subplot(gs[3], sharex=ax1)
    ax4.plot(df.index, df['Volatility'], color='#FFD166', linewidth=1.5)
    ax4.fill_between(df.index, df['Volatility'], color='#FFD166', alpha=0.2)
    ax4.set_title('20日历史波动率\n[数值升高预示市场焦虑，配合价格下跌需警惕恐慌抛售]', 
                color='white', fontsize=12, pad=12)

    # ------------------------------
    # 5. 日回报分布
    # ------------------------------
    ax5 = plt.subplot(gs[4], sharex=ax1)
    ax5.bar(df.index, df['Daily Return'], 
           color=np.where(df['Daily Return'] > 0, '#00E0A0', '#FF5E7E'),
           alpha=0.8, width=0.8)
    ax5.set_title('日收益率分布\n[反映价格变动的幅度和趋势]', 
                color='white', fontsize=12, pad=12)

    # ------------------------------
    # 6. 价格波动幅度（日内动能）
    # ------------------------------
    ax6 = plt.subplot(gs[5], sharex=ax1)
    ax6.plot(df.index, df['Range Pct'], color='#C792FF', linewidth=1.5)
    ax6.fill_between(df.index, df['Range Pct'], color='#C792FF', alpha=0.15,label="每日波动幅度")
    ax6.axhline(y=df["Range Pct"].mean(), linestyle="--", color="red", label="历史均值",alpha=0.5) #波动均值
    mean_range = df['Range Pct'].mean()
    std_range = df['Range Pct'].std()
    ax6.set_title(f'日内价格波动幅度（(High-Low)/Low）\n[波动率放大常预示变盘，配合成交量判断突破方向]\n'
             f'历史平均波动:{mean_range:.2f}% 标准差:{std_range:.2f}%', 
             color='white', fontsize=12, pad=12)
    ax6.legend(loc='upper left', fontsize=10)

    # 全局格式优化
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.grid(color='#363636', linestyle='--', linewidth=0.8)
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
        ax.tick_params(axis='both', colors='white')
    
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    plt.gcf().autofmt_xdate() 
    plt.tight_layout(pad = 0.1)
    return fig